In [1]:
import pandas as pd
train = pd.read_csv('./train.csv')
train_df = pd.DataFrame(train, columns = train.columns)#DataFrame
train_df.head(3)

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,Presence
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,Absence
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,Absence


In [2]:
train_data = train_df.iloc[:,:-1]
train_target = train_df.iloc[:,-1]

#target 데이터 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
target_encoded = le.fit_transform(train_target)
print(type(target_encoded),target_encoded)

<class 'numpy.ndarray'> [1 0 0 ... 1 1 0]


In [3]:
train_encoded = pd.DataFrame(train_data, columns = train.columns)
train_encoded['target'] = target_encoded
train_encoded.head(3)#absenece가 0, presence가 1

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease,target
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,NaN,1
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,NaN,0
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,NaN,0


In [22]:
train_encoded.drop('Heart Disease', axis = 1, inplace = True)#axis = 1은 필수
train_encoded.drop('id', axis = 1, inplace = True)

In [23]:
#데이터 전처리
train_encoded.isnull()

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,target
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629995,False,False,False,False,False,False,False,False,False,False,False,False,False,False
629996,False,False,False,False,False,False,False,False,False,False,False,False,False,False
629997,False,False,False,False,False,False,False,False,False,False,False,False,False,False
629998,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [24]:
train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630000 entries, 0 to 629999
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Age                      630000 non-null  int64  
 1   Sex                      630000 non-null  int64  
 2   Chest pain type          630000 non-null  int64  
 3   BP                       630000 non-null  int64  
 4   Cholesterol              630000 non-null  int64  
 5   FBS over 120             630000 non-null  int64  
 6   EKG results              630000 non-null  int64  
 7   Max HR                   630000 non-null  int64  
 8   Exercise angina          630000 non-null  int64  
 9   ST depression            630000 non-null  float64
 10  Slope of ST              630000 non-null  int64  
 11  Number of vessels fluro  630000 non-null  int64  
 12  Thallium                 630000 non-null  int64  
 13  target                   630000 non-null  int64  
dtypes: f

In [25]:
print(train_encoded.isnull().sum())

Age                        0
Sex                        0
Chest pain type            0
BP                         0
Cholesterol                0
FBS over 120               0
EKG results                0
Max HR                     0
Exercise angina            0
ST depression              0
Slope of ST                0
Number of vessels fluro    0
Thallium                   0
target                     0
dtype: int64


In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data, target_encoded, test_size = 0.2)

In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
dt = DecisionTreeClassifier(max_depth = 5, min_samples_leaf = 4)
dt.fit(X_train, y_train)
dt_predict = dt.predict(X_test)
print(accuracy_score(y_test,dt_predict))

0.862484126984127


In [28]:
from sklearn.model_selection import cross_val_score
import numpy as np
score = cross_val_score(dt, train_data, target_encoded, cv = 5)#전체 x, 전체 y를 집어넣어야 함
print(np.mean(score))

0.8626349206349205


In [29]:
from sklearn.model_selection import GridSearchCV
params = {'max_depth' : [3,5,7,9],
         'min_samples_leaf': [2,3,4,5]}
grid_dt = GridSearchCV(dt, param_grid = params, cv = 5)
grid_dt.fit(X_train, y_train)
print(grid_dt.best_params_)
best_model = grid_dt.predict(X_test)
print(accuracy_score(y_test,best_model))
'''pred = grid_dt.predict(X_test) 이것만으로도 가능(간단)
best_model = grid_dt.best_estimator_
pred = best_model.predict(X_test)이 원래는 정석'''

{'max_depth': 9, 'min_samples_leaf': 5}
0.8801587301587301


'pred = grid_dt.predict(X_test) 이것만으로도 가능(간단)\nbest_model = grid_dt.best_estimator_\npred = best_model.predict(X_test)이 원래는 정석'

In [30]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 300, max_depth = 9)#n_estimators 추가
rfc.fit(X_train, y_train)
rfc_dt = rfc.predict(X_test)
print(accuracy_score(y_test, rfc_dt))

0.8840555555555556


In [31]:
from xgboost import XGBClassifier#XGBoostClassifier아님
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)
xgb = XGBClassifier(n_estimators = 500, learning_rate = 0.1, max_depth = 3, eval_metric = 'logloss')#early_stopping_rounds = 100
'''원래는 early_stopping_rounds를 썼지만 voting을 위해 제거'''
xgb.fit(X_train,y_train, eval_set=[(X_val, y_val)])#early_stopping_rounds, eval_metric은 위로
xgb_predict = xgb.predict(X_test)
print(accuracy_score(y_test,xgb_predict))

[0]	validation_0-logloss:0.63743
[1]	validation_0-logloss:0.59613
[2]	validation_0-logloss:0.56152
[3]	validation_0-logloss:0.53239
[4]	validation_0-logloss:0.50739
[5]	validation_0-logloss:0.48590
[6]	validation_0-logloss:0.46712
[7]	validation_0-logloss:0.45056
[8]	validation_0-logloss:0.43604
[9]	validation_0-logloss:0.42309
[10]	validation_0-logloss:0.41107
[11]	validation_0-logloss:0.40026
[12]	validation_0-logloss:0.39045
[13]	validation_0-logloss:0.38151
[14]	validation_0-logloss:0.37346
[15]	validation_0-logloss:0.36627
[16]	validation_0-logloss:0.35958
[17]	validation_0-logloss:0.35350
[18]	validation_0-logloss:0.34806
[19]	validation_0-logloss:0.34306
[20]	validation_0-logloss:0.33845
[21]	validation_0-logloss:0.33421
[22]	validation_0-logloss:0.33029
[23]	validation_0-logloss:0.32664
[24]	validation_0-logloss:0.32324
[25]	validation_0-logloss:0.32014
[26]	validation_0-logloss:0.31723
[27]	validation_0-logloss:0.31455
[28]	validation_0-logloss:0.31201
[29]	validation_0-loglos

In [32]:
from lightgbm import LGBMClassifier
from lightgbm import early_stopping, log_evaluation#버전이 바뀌면서 미리 지정해 줘야함
lgbm = LGBMClassifier(n_estimators = 500, learning_rate = 0.1)
lgbm.fit(X_train, y_train, eval_set = [(X_val, y_val)], eval_metric = 'logloss', callbacks = [log_evaluation(10)])# callbacks[early_stopping(stopping_rounds = 100)]
'''early_stopping_rounds와 log_evaluation은 callback안에 작성, eval_set안은 []안에 한번 더 ()로 묶어줘야 함, 원래는 early_stopping을 썼지만 voting을 위해 제거'''
lgb_predict = lgbm.predict(X_test)
print(accuracy_score(y_test,lgb_predict))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 180985, number of negative: 222215
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023303 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 670
[LightGBM] [Info] Number of data points in the train set: 403200, number of used features: 14
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.448872 -> initscore=-0.205231
[LightGBM] [Info] Start training from score -0.205231
[10]	valid_0's binary_logloss: 0.390536
[20]	valid_0's binary_logloss: 0.314978
[30]	valid_0's binary_logloss: 0.288868
[40]	valid_0's binary_logloss: 0.278881
[50]	valid_0's binary_logloss: 0.274764
[60]	valid_0's binary_logloss: 0.272815
[70]	valid_0's binary_logloss: 0.271811


In [33]:
from sklearn.ensemble import VotingClassifier
vt = VotingClassifier(estimators = [('xgboost', xgb),('lightgbm',lgbm)], voting = 'soft')#estimators도 []안에 ()로 각각 묶음
vt.fit(X_train, y_train)
vt_predict = vt.predict(X_test)
print(accuracy_score(y_test, vt_predict))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 180985, number of negative: 222215
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022884 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 670
[LightGBM] [Info] Number of data points in the train set: 403200, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.448872 -> initscore=-0.205231
[LightGBM] [Info] Start training from score -0.205231
0.888515873015873


In [34]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()
params = {'num_leaves':[32,64], 'max_depth':[10,20], 'n_estimators':[100,200,300,400] }#[]로 감싸야 함
lgbm_grid = GridSearchCV(lgbm, param_grid = params,cv = 3)
lgbm_grid.fit(X_train, y_train)
lgbm_grid_predict = lgbm_grid.predict(X_test)
print(accuracy_score(y_test, lgbm_grid_predict))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 120657, number of negative: 148143
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016971 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 670
[LightGBM] [Info] Number of data points in the train set: 268800, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.448873 -> initscore=-0.205226
[LightGBM] [Info] Start training from score -0.205226
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 120657, number of negative: 148143
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018950 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `for

In [38]:
test = pd.read_csv("test.csv")
'''final_pred = lgbm_grid.predict(test)처럼 통째로 넣으면 안됨'''
submission_id = test['id']#id를 미리 따로 빼서 저장 >> inplace = True를 안하면 굳이 임
test_input = test.drop('id', axis = 1)
'''from sklearn.preprocessing import LabelEncoder >> test에는 heart disease컬럼이 없으므로 인코딩 필요 x'''
final_pred = lgbm_grid.predict(test)

In [43]:
# 딕셔너리 형태로 묶어서 DataFrame을 만듭니다.
submission = pd.DataFrame({
    'id': submission_id,          # 원본 데이터에서 id만 쏙 가져오기
    'Heart Disease': final_pred      # 방금 만든 예측값 변수 넣기, 같은 모델로 inverse_transform()인줄 알았으나 숫자형으로 들어감
})

# 확인해보기 (이미지랑 똑같이 생겼는지)
print(submission.head())

       id  Heart Disease
0  630000              1
1  630001              0
2  630002              1
3  630003              0
4  630004              0


In [44]:
submission.to_csv('submission.csv', index = False)# .to_csv로 csv파일로 변경, index = false는 필수임

In [45]:
submission_csv = pd.read_csv('submission.csv')
submission_csv.head(3)

,id,Heart Disease
0,630000,1
1,630001,0
2,630002,1


In [83]:
from autogluon.tabular import TabularPredictor

test = pd.read_csv("test.csv")
# 1. 학습 (train 데이터 통째로 넣고, 정답 컬럼 이름만 알려주면 끝) >> 무조건 원본 데이터
# time_limit : 몇 초 동안 학습할지 (길수록 성능 좋아짐, 예: 600초)
predictor = TabularPredictor(label='Heart Disease', eval_metric='accuracy').fit(
    train_data=train, 
    time_limit=600, 
    presets='best_quality')

# 2. 리더보드 확인 (어떤 모델이 1등 했는지 보여줌)
predictor.leaderboard(train)

# 3. 예측 (test 데이터 넣기)
pred_y = predictor.predict(test)


No path specified. Models will be saved in: "AutogluonModels\ag-20260216_142742"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.13.5
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          8
Pytorch Version:    2.9.1+cpu
CUDA Version:       CUDA is not available
Memory Avail:       6.00 GB / 15.73 GB (38.2%)
Disk Space Avail:   286.70 GB / 459.68 GB (62.4%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	Th

KeyError: 'Heart Disease'

In [78]:
#EDA
train_encoded.head()

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,target
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,1
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,0
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,0
3,3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,0
4,4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,1
